In [ ]:
### This file was used to add the downloaded ERA5 data to impute the missing FLUXNET data

# ERA5 data was sourced in file 3 and processed in file 4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
 


In [ ]:
### fill in the missing data with ERA5 land ###

In [ ]:
# import the df from  2.missing_data_exploration

df=pd.read_csv('/Users/abigailbase/PROJECT FILES/Pre imputation data/pre_imp.csv',index_col=0,parse_dates=['date'])

In [ ]:
#import csvs

ERA5_AR_Vir=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/AR-Vir.csv',parse_dates=['valid_time'])
ERA5_AU_Dry=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/AU-Dry.csv',parse_dates=['valid_time'])
ERA5_BE_Vie=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/BE-Vie.csv',parse_dates=['valid_time'])
ERA5_CA_TP1=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/CA-TP1.csv',parse_dates=['valid_time'])
ERA5_CH_Cha=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/CH-Cha.csv',parse_dates=['valid_time'])
ERA5_DE_Gri=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/DE-Gri.csv',parse_dates=['valid_time'])
ERA5_FR_Pue=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/FR-Pue.csv',parse_dates=['valid_time'])
ERA5_GF_Guy=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/GF-Guy.csv',parse_dates=['valid_time'])
ERA5_IT_Col=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/IT-Col.csv',parse_dates=['valid_time'])
ERA5_NL_Loo=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/NL-Loo.csv',parse_dates=['valid_time'])
ERA5_RU_Cok=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/RU-Cok.csv',parse_dates=['valid_time'])
ERA5_RU_Fyo=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/RU-Fyo.csv',parse_dates=['valid_time'])
ERA5_US_PFa=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/US-PFa.csv',parse_dates=['valid_time'])
ERA5_US_Var=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/US-Var.csv',parse_dates=['valid_time'])
ERA5_ZA_Kru=pd.read_csv('/Users/abigailbase/PROJECT FILES/ERA5 CSVs/ZA-Kru.csv',parse_dates=['valid_time'])


In [ ]:
# prepare the ERA5 data for imputing by converting the units 
# and creating required variables

In [ ]:
def process_era5_data(df):
    
    # Convert temperatures to Celsius from Kelvin
    df['t2m'] = df['t2m'] - 273.15  # temp
    df['d2m'] = df['d2m'] - 273.15  # dpt
    
    # Convert pressure to kPa from Pa
    df['sp'] = df['sp'] / 1000
    
    # Calculate Saturation Vapor Pressure (svp) in kPa
    def svp(T): 
        return 0.6112 * np.exp((17.67 * T) / (T + 243.5))
    
    # Calculate VPD in hPa
    def calc_vpd(temp, DPT):
        svp_temp = svp(temp)
        svp_dpt = svp(DPT)
        vpd_kPa = svp_temp - svp_dpt
        vpd_hPa = vpd_kPa * 10  # convert to hPa
        return vpd_hPa
    
    df['VPD'] = df.apply(lambda row: calc_vpd(row['t2m'], row['d2m']), axis=1)
    
    # Convert SSRD from J/m² to W/m²
    df['ssrd'] = df['ssrd'] / 86400
    
    # Convert SSRD to PPFD
    kEC = 2.04  # conversion factor
    df['PPFD'] = df['ssrd'] * kEC
    
    # Convert precipitation from m to mm
    df['tp'] = df['tp'] * 1000
    
    # Calculate wind speed
    df['wind_speed'] = np.sqrt(df['u10']**2 + df['v10']**2)
    
    # Convert soil water content to percentage
    df['swvl1'] = df['swvl1'] * 100
    
    # Convert soil temp (tsn) to Celcius
    df['tsn']=df['tsn']- 273.15
    
    return df


In [ ]:
def ppfd_process(df):
    
    # Convert SSRD from J/m² to W/m²
    df['ssrd'] = df['ssrd'] / 86400
    
    # Convert SSRD to PPFD
    kEC = 2.04  # conversion factor
    
    return df

In [ ]:
def swc_process(df):
    # Convert soil water content to percentage
    df['swvl1'] = df['swvl1'] * 100
    
    return df

In [ ]:
def st_process(df):
    # Convert soil temp to deg c
    df['tsn']=df['tsn']- 273.15 
    
    return df
    

In [ ]:
# apply the required processing (See report appendix)

In [ ]:
ERA5_AR_Vir = process_era5_data(ERA5_AR_Vir) #all variables

In [ ]:
ERA5_AU_Dry=ppfd_process(ERA5_AU_Dry) #ppfd only

In [ ]:
ERA5_BE_Vie=ppfd_process(ERA5_BE_Vie) #ppfd only

In [ ]:
ERA5_CA_TP1=ppfd_process(ERA5_CA_TP1) #ppfd only

In [ ]:
ERA5_CH_Cha=ppfd_process(ERA5_CH_Cha) #ppfd only

In [ ]:
ERA5_DE_Gri=ppfd_process(ERA5_DE_Gri) #ppfd only

In [ ]:
ERA5_FR_Pue=ppfd_process(ERA5_FR_Pue) 
ERA5_FR_Pue=swc_process(ERA5_FR_Pue) #ppfd and swc

In [ ]:
ERA5_GF_Guy=ppfd_process(ERA5_GF_Guy)
ERA5_GF_Guy=st_process(ERA5_GF_Guy) #soil temp and ppfd

In [ ]:
ERA5_IT_Col=ppfd_process(ERA5_IT_Col)
ERA5_IT_Col=swc_process(ERA5_IT_Col) #ppfd and swc

In [ ]:
ERA5_NL_Loo=ppfd_process(ERA5_NL_Loo) #ppfd

In [ ]:
ERA5_RU_Cok=ppfd_process(ERA5_RU_Cok)
ERA5_RU_Cok=swc_process(ERA5_RU_Cok)
ERA5_RU_Cok=st_process(ERA5_RU_Cok) #ppfd, swc st

In [ ]:
ERA5_RU_Fyo=ppfd_process(ERA5_RU_Fyo)

In [ ]:
ERA5_US_PFa=ppfd_process(ERA5_US_PFa) 
ERA5_US_PFa=st_process(ERA5_US_PFa) #ppfd, st

In [ ]:
ERA5_US_Var=ppfd_process(ERA5_US_Var) #ppfd only


In [ ]:
ERA5_ZA_Kru = process_era5_data(ERA5_ZA_Kru) #all variables

In [ ]:
#seperate out the data frames by site_id

AR_Vir=df[df['SITE_ID']=='AR-Vir']#1
AU_Dry=df[df['SITE_ID']=='AU-Dry']#2
BE_Vie=df[df['SITE_ID']=='BE-Vie']#3
CA_TP1=df[df['SITE_ID']=='CA-TP1']#4
CH_Cha=df[df['SITE_ID']=='CH-Cha']#5
DE_Gri=df[df['SITE_ID']=='DE-Gri']#6
FR_Pue=df[df['SITE_ID']=='FR-Pue']#7
GF_Guy=df[df['SITE_ID']=='GF-Guy']#8
IT_Col=df[df['SITE_ID']=='IT-Col']#9
NL_Loo=df[df['SITE_ID']=='NL-Loo']#10
RU_Cok=df[df['SITE_ID']=='RU-Cok']#11
RU_Fyo=df[df['SITE_ID']=='RU-Fyo']#12
US_PFa=df[df['SITE_ID']=='US-PFa']#13
US_Var=df[df['SITE_ID']=='US-Var']#14
ZA_Kru=df[df['SITE_ID']=='ZA-Kru']#15



In [ ]:
### now fill in the missing data from each site ###

In [ ]:
# AR-Vir

In [ ]:
# want to impute, TA_F, PA_F,VPD_F,P_F,WS_F,TS_F_MDS_1,SWC_F_MDS_1


In [ ]:
ERA5_AR_Vir=ERA5_AR_Vir.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_AR_Vir.set_index('valid_time',inplace=True) # set index 

In [ ]:
AR_Vir.set_index('date',inplace=True)# set index 

In [ ]:
# dictionary for renaming the columns to have same names

ERA5_renames={'t2m':'TA_F','sp':'PA_F','ssrd':'PPFD_IN','tp':'P_F',
              'tsn':'TS_F_MDS_1','swvl1':'SWC_F_MDS_1','wind_speed':'WS_F',
             'VPD':'VPD_F'}

In [ ]:
ERA5_AR_Vir.rename(columns=ERA5_renames,inplace=True)

In [ ]:
## from this need to drop Co2 GPP, NEE as this data needs to come from
## somewhere else from AR_Vir

In [ ]:
AR_Vir_drop=['CO2_F_MDS','CO2_F_MDS','GPP_DT_VUT_REF','NEE_VUT_REF']

In [ ]:
AR_Vir=AR_Vir.drop(columns=AR_Vir_drop)

In [ ]:
# now impute with ERA5 land data

for column in AR_Vir.columns:
    
    # iterate  rows
    
    for index, value in AR_Vir[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            # Impute the missing value with the corresponding value from ERA_AR_Vir
            AR_Vir.at[index, column] = ERA5_AR_Vir.at[index, column]




In [ ]:
AR_Vir.shape

In [ ]:
AR_Vir.isna().sum()

In [ ]:
### AU-Dry (PPFD) ###

In [ ]:
ERA5_AU_Dry=ERA5_AU_Dry.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_AU_Dry.set_index('valid_time',inplace=True) # set index 

In [ ]:
AU_Dry.set_index('date',inplace=True) # set index 

In [ ]:
ERA5_AU_Dry.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in AU_Dry.columns:
    
    # iterate  rows
    
    for index, value in AU_Dry[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            AU_Dry.at[index, column] = ERA5_AU_Dry.at[index, column]



In [ ]:
AU_Dry.isna().sum()

In [ ]:
### BE-Vie (PPFD) ###

ERA5_BE_Vie=ERA5_BE_Vie.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_BE_Vie.set_index('valid_time',inplace=True) # set index 

In [ ]:
BE_Vie.set_index('date',inplace=True) # set index 

In [ ]:
ERA5_BE_Vie.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in BE_Vie.columns:
    
    # iterate  rows
    
    for index, value in BE_Vie[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            BE_Vie.at[index, column] = ERA5_BE_Vie.at[index, column]


In [ ]:
BE_Vie.isna().sum()

In [ ]:
BE_Vie.shape

In [ ]:
### CA-TP1 (PPFD) ###

ERA5_CA_TP1=ERA5_CA_TP1.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_CA_TP1.set_index('valid_time',inplace=True) # set index 

In [ ]:
CA_TP1.set_index('date',inplace=True) # set index 

In [ ]:
ERA5_CA_TP1.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in CA_TP1.columns:
    
    # iterate  rows
    
    for index, value in CA_TP1[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            CA_TP1.at[index, column] = ERA5_CA_TP1.at[index, column]


In [ ]:
CA_TP1.isna().sum()

In [ ]:
### CH-Cha (PPFD) ###

ERA5_CH_Cha=ERA5_CH_Cha.sort_values(by='valid_time') # sort date


In [ ]:
ERA5_CH_Cha.set_index('valid_time',inplace=True)

In [ ]:
CH_Cha.set_index('date',inplace=True) # set index 

In [ ]:
ERA5_CH_Cha.rename(columns=ERA5_renames,inplace=True) # rename cols


In [ ]:
for column in CH_Cha.columns:
    
    # iterate  rows
    
    for index, value in CH_Cha[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            CH_Cha.at[index, column] = ERA5_CH_Cha.at[index, column]


In [ ]:
CH_Cha.isna().sum()

In [ ]:
### DE-Gri (PPFD) ###

ERA5_DE_Gri=ERA5_DE_Gri.sort_values(by='valid_time') # sort date


In [ ]:
ERA5_DE_Gri.set_index('valid_time',inplace=True) #set index

In [ ]:
DE_Gri.set_index('date',inplace=True) #set index

In [ ]:
ERA5_DE_Gri.rename(columns=ERA5_renames,inplace=True) # rename cols


In [ ]:
for column in DE_Gri.columns:
    
    # iterate  rows
    
    for index, value in DE_Gri[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            DE_Gri.at[index, column] = ERA5_DE_Gri.at[index, column]


In [ ]:
DE_Gri.isna().sum()

In [ ]:
### FR-Pue (PPFD, SWC)

In [ ]:
ERA5_FR_Pue=ERA5_FR_Pue.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_FR_Pue.set_index('valid_time',inplace=True) #set index

In [ ]:
FR_Pue.set_index('date',inplace=True) #set index

In [ ]:
ERA5_FR_Pue.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in FR_Pue.columns:
    
    # iterate  rows
    
    for index, value in FR_Pue[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            FR_Pue.at[index, column] = ERA5_FR_Pue.at[index, column]


In [ ]:
FR_Pue.isna().sum()

In [ ]:
### GF-Guy ###

In [ ]:
ERA5_GF_Guy=ERA5_GF_Guy.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_GF_Guy.set_index('valid_time',inplace=True) #set index

In [ ]:
GF_Guy.set_index('date',inplace=True) #set index

In [ ]:
ERA5_GF_Guy.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in GF_Guy.columns:
    
    # iterate  rows
    
    for index, value in GF_Guy[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            GF_Guy.at[index, column] = ERA5_GF_Guy.at[index, column]


In [ ]:
### IT-Col ###

In [ ]:
ERA5_IT_Col=ERA5_GF_Guy.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_IT_Col.set_index('valid_time',inplace=True) #set index

In [ ]:
IT_Col.set_index('date',inplace=True) #set index

In [ ]:
ERA5_IT_Col.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in IT_Col.columns:
    
    # iterate  rows
    
    for index, value in IT_Col[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            IT_Col.at[index, column] = ERA5_IT_Col.at[index, column]


In [ ]:
IT_Col.isna().sum()

In [ ]:
### NL-Loo ###

In [ ]:
ERA5_NL_Loo=ERA5_GF_Guy.sort_values(by='valid_time') # sort date

In [ ]:
NL_Loo.set_index('date',inplace=True) #set index

In [ ]:
ERA5_NL_Loo.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
NL_Loo.isna().sum()

In [ ]:
NL_Loo=NL_Loo.drop(columns='CO2_F_MDS')

In [ ]:
for column in NL_Loo.columns:
    
    # iterate  rows
    
    for index, value in NL_Loo[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            NL_Loo.at[index, column] = ERA5_NL_Loo.at[index, column]


In [ ]:
NL_Loo.isna().sum()

In [ ]:
### RU-Cok ###

In [ ]:
ERA5_RU_Cok=ERA5_RU_Cok.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_RU_Cok.set_index('valid_time',inplace=True) # set index 

In [ ]:
RU_Cok.set_index('date',inplace=True)# set index 

In [ ]:
ERA5_RU_Cok.rename(columns=ERA5_renames,inplace=True)

In [ ]:
## from this need to drop Co2 GPP, NEE as this data needs to come from
## somewhere else from AR_Vir

In [ ]:
AR_Vir_drop=['CO2_F_MDS','CO2_F_MDS','GPP_DT_VUT_REF','NEE_VUT_REF']

In [ ]:
RU_Cok=RU_Cok.drop(columns=AR_Vir_drop)

In [ ]:
# now impute with ERA5 land data

for column in RU_Cok.columns:
    
    # iterate  rows
    
    for index, value in RU_Cok[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            # Impute the missing value with the corresponding value from ERA_AR_Vir
            RU_Cok.at[index, column] = ERA5_RU_Cok.at[index, column]




In [ ]:
RU_Cok.isna().sum()

In [ ]:
## RU-Fyo ###

In [ ]:
ERA5_RU_Fyo=ERA5_RU_Fyo.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_RU_Fyo.set_index('valid_time',inplace=True) #set index

In [ ]:
RU_Fyo.set_index('date',inplace=True) #set index

In [ ]:
ERA5_RU_Fyo.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in RU_Fyo.columns:
    
    # iterate  rows
    
    for index, value in RU_Fyo[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            RU_Fyo.at[index, column] = ERA5_RU_Fyo.at[index, column]


In [ ]:
RU_Fyo.isna().sum()

In [ ]:
### US-PFa ###

In [ ]:
ERA5_US_PFa

In [ ]:
ERA5_US_PFa=ERA5_US_PFa.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_US_PFa.set_index('valid_time',inplace=True) #set index

In [ ]:
US_PFa.set_index('date',inplace=True) #set index

In [ ]:
ERA5_US_PFa.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in US_PFa.columns:
    
    # iterate  rows
    
    for index, value in US_PFa[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            US_PFa.at[index, column] = ERA5_US_PFa.at[index, column]


In [ ]:
US_PFa.isna()

In [ ]:
### US-Var ###

In [ ]:
ERA5_US_Var=ERA5_US_Var.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_US_Var.set_index('valid_time',inplace=True) #set index

In [ ]:
US_Var.set_index('date',inplace=True) #set index

In [ ]:
ERA5_US_Var.rename(columns=ERA5_renames,inplace=True) # rename cols

In [ ]:
for column in US_Var.columns:
    
    # iterate  rows
    
    for index, value in US_Var[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            US_Var.at[index, column] = ERA5_US_Var.at[index, column]


In [ ]:
US_Var.isna().sum()

In [ ]:
### ZA-Kru ###

In [ ]:
ERA5_ZA_Kru=ERA5_ZA_Kru.sort_values(by='valid_time') # sort date

In [ ]:
ERA5_ZA_Kru.set_index('valid_time',inplace=True) # set index 

In [ ]:
ZA_Kru.set_index('date',inplace=True)# set index 

In [ ]:
ERA5_ZA_Kru.rename(columns=ERA5_renames,inplace=True)

In [ ]:
## from this need to drop Co2 GPP, NEE as this data needs to come from
## somewhere else from AR_Vir

In [ ]:
ZA_Kru=ZA_Kru.drop(columns=AR_Vir_drop)

In [ ]:
# now impute with ERA5 land data

for column in ZA_Kru.columns:
    
    # iterate  rows
    
    for index, value in ZA_Kru[column].items():
        
        # check for NA values
        
        if pd.isna(value):
            
            # Impute the missing value with the corresponding value from ERA_AR_Vir
            ZA_Kru.at[index, column] = ERA5_ZA_Kru.at[index, column]




In [ ]:
ZA_Kru.isna().sum()

In [ ]:
### i think for the dropped columns i will reseperate the DFs 
### and keep the date to impute them and then join right at the end
### on the date column

In [ ]:
### import co2 data

In [ ]:
co2_2010=pd.read_csv('/Users/abigailbase/PROJECT FILES/CO2 CSVs/co2_2010.csv',index_col=0)
co2_2011=pd.read_csv('/Users/abigailbase/PROJECT FILES/CO2 CSVs/co2_2011.csv',index_col=0)
co2_2012=pd.read_csv('/Users/abigailbase/PROJECT FILES/CO2 CSVs/co2_2012.csv',index_col=0)
co2_2013=pd.read_csv('/Users/abigailbase/PROJECT FILES/CO2 CSVs/co2_2013.csv',index_col=0)
co2_2014=pd.read_csv('/Users/abigailbase/PROJECT FILES/CO2 CSVs/co2_2014.csv',index_col=0)



In [ ]:
co2_2010['time']=pd.to_datetime(co2_2010['time']).dt.date
co2_2011['time']=pd.to_datetime(co2_2011['time']).dt.date
co2_2012['time']=pd.to_datetime(co2_2012['time']).dt.date
co2_2013['time']=pd.to_datetime(co2_2013['time']).dt.date
co2_2014['time']=pd.to_datetime(co2_2014['time']).dt.date

In [ ]:
co2_drop=['level','LAT','LON']

In [ ]:
co2_2010=co2_2010.drop(columns=co2_drop)
co2_2011=co2_2011.drop(columns=co2_drop)
co2_2012=co2_2012.drop(columns=co2_drop)
co2_2013=co2_2013.drop(columns=co2_drop)
co2_2014=co2_2014.drop(columns=co2_drop)

In [ ]:
#combine the years into one


co2=pd.concat([co2_2010,co2_2011,co2_2012,co2_2013,co2_2014],ignore_index=True)

In [ ]:
AR_Vir_co2=co2[co2['site_id']=='AR-Vir']
RU_Cok_co2=co2[co2['site_id']=='RU-Cok']
NL_Loo_co2=co2[co2['site_id']=='NL-Loo']
ZA_Kru_co2=co2[co2['site_id']=='ZA-Kru']

In [ ]:
NL_Loo_co2=co2[co2['site_id']=='NL-Loo']

In [ ]:
AR_Vir_co2

In [ ]:
co2_rename={'co2':'CO2_F_MDS'}

In [ ]:
AR_Vir_co2=AR_Vir_co2.set_index('time')

In [ ]:
AR_Vir_co2.rename(columns=co2_rename,inplace=True)

In [ ]:
AR_Vir_co2

In [ ]:
### need to impute co2 for RU-Cok, AR-Vir and ZA_Kru

AR_Vir

In [ ]:
### AR-Vir ###

In [ ]:
# want to join AR_Vir_co2 and AR_Vir

In [ ]:
AR_Vir_imp=AR_Vir.join(AR_Vir_co2)

In [ ]:
AR_Vir_imp=AR_Vir_imp.drop(columns='site_id')

In [ ]:
AR_Vir_imp.isna().sum()

In [ ]:
## fill missing 12 with mean DELETE THIS

In [ ]:
AR_Vir_imp['CO2_F_MDS']=AR_Vir_imp['CO2_F_MDS'].fillna(AR_Vir_imp['CO2_F_MDS'].mean())

In [ ]:
AR_Vir_imp.isna().sum()

In [ ]:
### RU-Cok ###

In [ ]:
RU_Cok_co2=RU_Cok_co2.set_index('time')

In [ ]:
RU_Cok_co2.rename(columns=co2_rename,inplace=True)

In [ ]:
RU_Cok_imp=RU_Cok.join(RU_Cok_co2)

In [ ]:
RU_Cok_imp['CO2_F_MDS']=RU_Cok_imp['CO2_F_MDS'].fillna(RU_Cok_imp['CO2_F_MDS'].mean())

In [ ]:
### NL_Loo ###

In [ ]:
NL_Loo_original=df[df['SITE_ID']=='NL-Loo']

In [ ]:
NL_Loo_original=NL_Loo_original[['CO2_F_MDS','date']]

In [ ]:
NL_Loo_original=NL_Loo_original.set_index('date')

In [ ]:
NL_Loo_original.isna().sum()

In [ ]:
NL_Loo_original['CO2_F_MDS']=NL_Loo_original['CO2_F_MDS'].fillna(NL_Loo_original['CO2_F_MDS'].mean())

In [ ]:
NL_Loo_original.isna().sum()

In [ ]:
NL_Loo_imp=NL_Loo.join(NL_Loo_original)

In [ ]:
NL_Loo_imp.isna().sum()

In [ ]:
NL_Loo_imp.head()

In [ ]:
NL_Loo_imp.isna().sum()

In [ ]:
### ZA-Kru ###

In [ ]:
ZA_Kru_co2=ZA_Kru_co2.set_index('time')

In [ ]:
ZA_Kru_co2.rename(columns=co2_rename,inplace=True)

In [ ]:
ZA_Kru_imp=ZA_Kru.join(AR_Vir_co2)

In [ ]:
ZA_Kru_imp.isna().sum()

In [ ]:
ZA_Kru_imp['CO2_F_MDS']=ZA_Kru_imp['CO2_F_MDS'].fillna(ZA_Kru_imp['CO2_F_MDS'].mean())

In [ ]:
ZA_Kru_imp=ZA_Kru_imp.drop(columns='site_id')

---

In [ ]:
### NIRv data from file 5 (hdf process) was also added here

In [ ]:
#import 

In [ ]:
AR_Vir_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/AR_Vir_NIR.csv')
AU_Dry_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/AU_Dry_NIRv.csv')
BE_Vie_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/BE_Vie_NIR.csv')
CA_TP1_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/CA_TP1_NIR.csv')
CH_Cha_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/CH_Cha_NIR.csv')
DE_Gri_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/DE_Gri_NIR.csv')
FR_Pue_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/FR_Pue_NIR.csv')
GF_Guy_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/GF_Guy_NIR.csv')

NL_Loo_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/NL_Loo_NIR.csv')
RU_Cok_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/RU_CokNIR.csv')
RU_Fyo_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/RU_Fyo_NIR.csv')
US_PFa_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/US_PFa_NIR.csv')
US_Var_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/US_Var_NIR.csv')
ZA_Kru_NIRv=pd.read_csv('/Users/abigailbase/PROJECT FILES/NIRv DFs/ZA_Kru_NIR.csv')



In [ ]:
file_paths = [
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/AR_Vir_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/AU_Dry_NIRv.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/BE_Vie_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/CA_TP1_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/CH_Cha_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/DE_Gri_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/FR_Pue_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/GF_Guy_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/IT_Col_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/NL_Loo_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/RU_CokNIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/RU_Fyo_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/US_PFa_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/US_Var_NIR.csv',
    '/Users/abigailbase/PROJECT FILES/NIRv DFs/ZA_Kru_NIR.csv'
]


In [ ]:
keys = [
    'AR_Vir_NIRv', 'AU_Dry_NIRv', 'BE_Vie_NIRv', 'CA_TP1_NIRv', 'CH_Cha_NIRv',
    'DE_Gri_NIRv', 'FR_Pue_NIRv', 'GF_Guy_NIRv','IT_Col_NIRv', 'NL_Loo_NIRv', 'RU_Cok_NIRv',
    'RU_Fyo_NIRv', 'US_PFa_NIRv', 'US_Var_NIRv', 'ZA_Kru_NIRv'
]


In [ ]:
nirv_dict = {}

In [ ]:
for key, file_path in zip(keys, file_paths):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    # Add the site_id column
    df['site_id'] = key
    # Store the DataFrame in the dictionary
    nirv_dict[key] = df


In [ ]:
### analyse trend to see which interpolation most ideal

dfs_to_concat = [nirv_dict[key] for key in keys]

In [ ]:
combined_nirv_df = pd.concat(dfs_to_concat, ignore_index=True)


In [ ]:
combined_nirv_df['date'] = pd.to_datetime(combined_nirv_df['date'])


In [ ]:
combined_nirv_df = combined_nirv_df.sort_values(by='date')


In [ ]:
sites = combined_nirv_df['site_id'].unique()


In [ ]:
#  time series of each site revealed seasonality so a cubic spline interpolator was appropriate

plt.figure(figsize=(14, 7))

for site in combined_nirv_df['site_id'].unique():
    site_data = combined_nirv_df[combined_nirv_df['site_id'] == site]
    plt.plot(site_data['date'], site_data['NIRv'], marker='o', linestyle='-', label=site)

plt.xlabel('Date')
plt.ylabel('NIRv')
plt.title('NIRv Time Series for All Sites')
plt.legend(title='Site ID')
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
AR_Vir_NIRv=nirv_dict['AR_Vir_NIRv']

In [ ]:
AR_Vir_NIRv

In [ ]:
NIRV_drop=['year','day_of_year','day_of_year','NIR_point','NDVI_point','site_id']

In [ ]:
AR_Vir_NIRv=AR_Vir_NIRv.drop(columns=NIRV_drop)

In [ ]:
AR_Vir_NIRv['date']=pd.to_datetime(AR_Vir_NIRv['date'])

In [ ]:
AR_Vir_NIRv.set_index('date', inplace=True)

In [ ]:
#create the missing days

AR_Vir_NIRv_daily = AR_Vir_NIRv.resample('D').asfreq()


In [ ]:
## apply the spine interpolator ###

valid_data = AR_Vir_NIRv_daily.dropna(subset=['NIRv'])

# convert dates to ordinal for interpolation
dates = valid_data.index.map(pd.Timestamp.toordinal).values
nirv_values = valid_data['NIRv'].values

cs = CubicSpline(dates, nirv_values, extrapolate=True)

full_dates = AR_Vir_NIRv_daily.index.map(pd.Timestamp.toordinal)
AR_Vir_NIRv_daily['NIRv'] = cs(full_dates)





In [ ]:

def interpolate_nirv(nirv_dict, key, NIRV_drop):
    AR_Vir_NIRv = nirv_dict[key]
    
    AR_Vir_NIRv = AR_Vir_NIRv.drop(columns=NIRV_drop)
    
    #  to datetime and set as index
    AR_Vir_NIRv['date'] = pd.to_datetime(AR_Vir_NIRv['date'])
    AR_Vir_NIRv.set_index('date', inplace=True)
    
    # drop data from before 2010
    AR_Vir_NIRv = AR_Vir_NIRv[AR_Vir_NIRv.index >= '2010-01-01']
    
    
    # daily frequency
    AR_Vir_NIRv_daily = AR_Vir_NIRv.resample('D').asfreq()
    
    valid_data = AR_Vir_NIRv_daily.dropna(subset=['NIRv'])
    
    # convert dates to ordinal for interpolation
    dates = valid_data.index.map(pd.Timestamp.toordinal).values
    nirv_values = valid_data['NIRv'].values
    
    # cubic spline interpolator
    cs = CubicSpline(dates, nirv_values, extrapolate=True)
    
    full_dates = AR_Vir_NIRv_daily.index.map(pd.Timestamp.toordinal)
    AR_Vir_NIRv_daily['NIRv'] = cs(full_dates)
    
    return AR_Vir_NIRv_daily



In [ ]:
#AR-Vir

AR_Vir_NIRv=nirv_dict['AR_Vir_NIRv']

In [ ]:
AR_Vir_NIRv_daily=interpolate_nirv(nirv_dict,'AR_Vir_NIRv',NIRV_drop)

In [ ]:
#AU-Dry

AU_Dry_NIRv=nirv_dict['AU_Dry_NIRv']

In [ ]:
AU_Dry_NIRv_daily=interpolate_nirv(nirv_dict,'AU_Dry_NIRv',NIRV_drop)

In [ ]:
#BE-Vie

BE_Vie_NIRv=nirv_dict['BE_Vie_NIRv']

In [ ]:
BE_Vie_NIRv_daily=interpolate_nirv(nirv_dict,'BE_Vie_NIRv',NIRV_drop)

In [ ]:
# CA-TP1

CA_TP1_NIRv=nirv_dict['CA_TP1_NIRv']

In [ ]:
CA_TP1_NIRv_daily=interpolate_nirv(nirv_dict,'CA_TP1_NIRv',NIRV_drop)

In [ ]:
# CH-Cha

CH_Cha_NIRv=nirv_dict['CH_Cha_NIRv']

In [ ]:
CH_Cha_NIRv_daily=interpolate_nirv(nirv_dict,'CH_Cha_NIRv',NIRV_drop)

In [ ]:
#DE-Gri

DE_Gri_NIRv=nirv_dict['DE_Gri_NIRv']

In [ ]:
DE_Gri_NIRv_daily=interpolate_nirv(nirv_dict,'DE_Gri_NIRv',NIRV_drop)

In [ ]:
#FR-Pue

FR_Pue_NIRv=nirv_dict['FR_Pue_NIRv']

In [ ]:
FR_Pue_NIRv_daily=interpolate_nirv(nirv_dict,'FR_Pue_NIRv',NIRV_drop)

In [ ]:
#GF-Guy

GF_Guy_NIRv=nirv_dict['GF_Guy_NIRv']

In [ ]:
GF_Guy_NIRv_daily=interpolate_nirv(nirv_dict,'GF_Guy_NIRv',NIRV_drop)

In [ ]:
#IT-Col

IT_Col_NIRv=nirv_dict['IT_Col_NIRv']

In [ ]:
IT_Col_NIRv_daily=interpolate_nirv(nirv_dict,'IT_Col_NIRv',NIRV_drop)

In [ ]:
# NL-Loo

In [ ]:
NL_Loo_NIRv=nirv_dict['NL_Loo_NIRv']

In [ ]:
NL_Loo_NIRv_daily=interpolate_nirv(nirv_dict,'NL_Loo_NIRv',NIRV_drop)

In [ ]:
NL_Loo_NIRv_daily.isna().sum()

In [ ]:
#RU-Cok

RU_Cok_NIRv=nirv_dict['RU_Cok_NIRv']

In [ ]:
RU_Cok_NIRv

In [ ]:
RU_Cok_NIRv['date']=pd.to_datetime(RU_Cok_NIRv['date'])

In [ ]:
RU_Cok_NIRv= RU_Cok_NIRv.sort_values(by='date')


In [ ]:
RU_Cok_NIRv=RU_Cok_NIRv.dropna()

In [ ]:
RU_Cok_NIRv=RU_Cok_NIRv.drop_duplicates(subset='date',keep='last')

In [ ]:
RU_Cok_NIRv.set_index('date', inplace=True)


In [ ]:
RU_Cok_NIRv = RU_Cok_NIRv[RU_Cok_NIRv.index >= '2010-01-01']
    

In [ ]:
RU_Cok_NIRv_daily = RU_Cok_NIRv.resample('D').asfreq()

In [ ]:
valid_data = RU_Cok_NIRv_daily.dropna(subset=['NIRv'])

dates = valid_data.index.map(pd.Timestamp.toordinal).values
nirv_values = valid_data['NIRv'].values

cs = CubicSpline(dates, nirv_values, extrapolate=True)
    
full_dates = RU_Cok_NIRv_daily.index.map(pd.Timestamp.toordinal)
RU_Cok_NIRv_daily['NIRv'] = cs(full_dates)
    
RU_Cok_NIRv_daily


In [ ]:
RU_Cok_imp

In [ ]:
RU_Cok_NIRv_daily.isna().sum()

In [ ]:
### RU-Fyo ###

In [ ]:
RU_Fyo_NIRv=nirv_dict['RU_Fyo_NIRv']

In [ ]:
RU_Fyo_NIRv_daily=interpolate_nirv(nirv_dict,'RU_Fyo_NIRv',NIRV_drop)

In [ ]:
### US-PFa ###

In [ ]:
US_PFa_NIRv=nirv_dict['US_PFa_NIRv']

In [ ]:
US_PFa_NIRv_daily=interpolate_nirv(nirv_dict,'RU_Fyo_NIRv',NIRV_drop)

In [ ]:
### US-Var ###

In [ ]:
US_Var_NIRv=nirv_dict['US_Var_NIRv']

In [ ]:
US_Var_NIRv_daily=interpolate_nirv(nirv_dict,'US_Var_NIRv',NIRV_drop)

In [ ]:
### ZA-Kru ###

In [ ]:
ZA_Kru_NIRv=nirv_dict['ZA_Kru_NIRv']

In [ ]:
ZA_Kru_NIRv_daily=interpolate_nirv(nirv_dict,'ZA_Kru_NIRv',NIRV_drop)

In [ ]:
### now add the NIRv data to the final df

In [ ]:
test=AR_Vir_imp.copy()

In [ ]:
test

In [ ]:
AR_Vir_imp['NIRv']=AR_Vir_NIRv_daily['NIRv']

In [ ]:
cutoff=pd.to_datetime('2014-12-19')

In [ ]:
# AR-Vir

AR_Vir_imp=AR_Vir[AR_Vir.index<=cutoff]

In [ ]:
AR_Vir_imp

In [ ]:
# AU_Dry

AU_Dry['NIRv']=AU_Dry_NIRv_daily['NIRv']

In [ ]:
AU_Dry=AU_Dry[AU_Dry.index<=cutoff]

In [ ]:
# BE-Vie

BE_Vie['NIRv']=BE_Vie_NIRv_daily['NIRv']

In [ ]:
BE_Vie=BE_Vie[BE_Vie.index<=cutoff]

In [ ]:
# CA-TP1

In [ ]:
CA_TP1['NIRv']=CA_TP1_NIRv_daily['NIRv']

In [ ]:
CA_TP1=CA_TP1[CA_TP1.index<=cutoff]

In [ ]:
# CH-CHa

In [ ]:
CH_Cha['NIRv']=CH_Cha_NIRv_daily['NIRv']

In [ ]:
CH_Cha=CH_Cha[CH_Cha.index<=cutoff]

In [ ]:
# DE-Gri

In [ ]:
DE_Gri['NIRv']=DE_Gri_NIRv_daily['NIRv']

In [ ]:
DE_Gri=DE_Gri[DE_Gri.index<=cutoff]

In [ ]:
# FR_Pue

In [ ]:
FR_Pue['NIRv']=FR_Pue_NIRv_daily['NIRv']

In [ ]:
FR_Pue=FR_Pue[FR_Pue.index<=cutoff]

In [ ]:
#GF_Guy

GF_Guy['NIRv']=GF_Guy_NIRv_daily['NIRv']

In [ ]:
GF_Guy=GF_Guy[GF_Guy.index<=cutoff]

In [ ]:
GF_Guy.isna().sum()

In [ ]:
# IT-Col

In [ ]:
IT_Col['NIRv']=IT_Col_NIRv_daily['NIRv']

In [ ]:
IT_Col=IT_Col[IT_Col.index<=cutoff]

In [ ]:
IT_Col.isna().sum()

In [ ]:
# NL-Loo

In [ ]:
NL_Loo

In [ ]:
NL_Loo_imp['NIRv']=NL_Loo_NIRv_daily['NIRv']

In [ ]:
NL_Loo_imp=NL_Loo[NL_Loo.index<=cutoff]

In [ ]:
NL_Loo_imp.isna().sum()

In [ ]:
# RU-Cok

In [ ]:
RU_Cok_imp['NIRv']=RU_Cok_NIRv_daily['NIRv']

In [ ]:
RU_Cok_imp=RU_Cok_imp[RU_Cok_imp.index<=cutoff]

In [ ]:
RU_Cok_imp.isna().sum()

In [ ]:
# RU_Fyo

In [ ]:
RU_Fyo['NIRv']=RU_Fyo_NIRv_daily['NIRv']

In [ ]:
RU_Fyo=RU_Fyo[RU_Fyo.index<=cutoff]

In [ ]:
# US-PFA

In [ ]:
US_PFa['NIRv']=US_PFa_NIRv_daily['NIRv']

In [ ]:
US_PFa=US_PFa[US_PFa.index<=cutoff]

In [ ]:
# US-Var

In [ ]:
US_Var['NIRv']=US_Var_NIRv_daily['NIRv']

In [ ]:
US_Var=US_Var[US_Var.index<=cutoff]

In [ ]:
# ZA-Kru

In [ ]:
ZA_Kru_imp['NIRv']=ZA_Kru_NIRv_daily['NIRv']

In [ ]:
ZA_Kru_imp=ZA_Kru[ZA_Kru.index<=cutoff]

In [ ]:
RU_Cok_imp=RU_Cok_imp.drop(columns='site_id')

In [ ]:
US_Var.shape

In [ ]:
# COMBINE INTO THE FINAL DF


final_df=pd.concat([AR_Vir_imp,AU_Dry,BE_Vie,CA_TP1,CH_Cha,DE_Gri,FR_Pue,GF_Guy,IT_Col,NL_Loo_imp,
                   RU_Cok_imp,RU_Fyo,US_PFa,US_Var,ZA_Kru_imp])

In [ ]:
RU_Fyo['NIRv']=RU_Fyo['NIRv'].fillna(RU_Fyo['NIRv'].mean())

In [ ]:
final_df.to_csv('/Users/abigailbase/PROJECT FILES/FINAL DFs/FINAL_FINAL_DF.csv',index=True)